In [18]:
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()
now = datetime.now()
formatted_date_time = now.strftime('%Y%m%d_%H%M')

In [19]:
def preprocess_film_metadata(df):
    df.fillna('', inplace=True)
    df['id'] = df['id'].astype(str)
    df['title'] = df['film_title'].str.lower()
    df['group_l1'] = df['group_name_l1'].str.lower()
    df['group_l2'] = df['group_name_l2'].str.lower()
    df['genres'] = df['film_genres'].apply(lambda x: ', '.join(x.split(',')))
    df['actors'] = df['film_actors'].apply(lambda x: ', '.join(x.split(',')))
    df['directors'] = df['film_directors'].apply(lambda x: ', '.join(x.split(',')))
    df.loc[df['actors'] == "various", 'actors'] = ""
    df['country'] = df['country_group'].str.lower()
    df['total_watchers'] = df['total_watchers'].astype('int')
    df['release_date'] = df['release_date'].str.replace(" 00:00:00", "")
    df['release_year'] = df['release_date'].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").year if x != '' else '')

    def popularity(total_watchers):
        if total_watchers >= 50000:
            return "trending"
        elif total_watchers < 50000 and total_watchers >= 500:
            return "average"
        else:
            return "below average"

    df['popularity'] = df['total_watchers'].apply(lambda x: popularity(x))
    search_text_columns = ['title', 'description', 'group_l1', 'group_l2', 'film_main_genre', 'genres', 'directors', 'actors', 'country', 'release_year', 'age_rating', 'popularity']
    df['search_text'] = df[search_text_columns].apply(lambda row: search_text(*row), axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    df.drop(columns=['film_title', 'group_name_l1', 'group_name_l2', 'film_main_genre', 'film_genres', 'film_directors', 'film_actors', 'country_group'], inplace=True)
    return df


def search_text(title, description, group_l1, group_l2, main_genre, genres, directors, actors, country, release_year, age_rating, popularity):
    return f"""title: {title}
description: {description}
group: {group_l1} > {group_l2}
genres: {main_genre}, {genres}
directors: {directors}
actors: {actors}
country: {country}
release year: {release_year}
age rating: {age_rating}
popularity: {popularity}"""

In [20]:
search_df = pd.read_excel('data/search_ground_truth.xlsx')
film_df = pd.read_excel('data/film_metadata.xlsx', converters={'release_date':str})

/home/tito/.venv/genai_hackathon_2024/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/tito/.venv/genai_hackathon_2024/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
film_df = preprocess_film_metadata(film_df)

In [22]:
film_df.head(2)

,id,description,release_date,total_watchers,age_rating,image_portrait,content_url,image_url,title,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text
0,7617,Menceritakan kisah seorang gadis bernama Sakin...,2023-03-13,294965,13 or more,bidadari-surgamu-2bbb68.jpg,https://www.vidio.com/premier/7617,https://thumbor.prod.vidiocdn.com/JxL1jUZmTFdc...,bidadari surgamu,series,tv sinetron,"drama, family, religi, romance","josephine firmstone, michelle joan, rizky naza...",anurag vaishnav,indonesia,2023,trending,title: bidadari surgamu\ndescription: Mencerit...
1,7576,"Naura (Basmalah), Rahsya (Raden Rakha), Adara ...",2023-02-23,269774,13 or more,magic-5-def33f.jpg,https://www.vidio.com/premier/7576,https://thumbor.prod.vidiocdn.com/kha0tcadN-wD...,magic 5,series,tv sinetron,"drama, fantasy, friendship","afan da5, basmalah, eby da 5, raden rakha, sri...","a. septian, bobby moeryawan, sondang pratama, ...",indonesia,2023,trending,"title: magic 5\ndescription: Naura (Basmalah),..."


In [23]:
joined_df = search_df.merge(film_df[['id', 'search_text']], left_on='query', right_on='id', how='left')
joined_df.drop(columns=['id'], inplace=True)
joined_df['search_text'] = np.where(joined_df['query_type'] == 'search', joined_df['query'], joined_df['search_text'])
joined_df['query'] = joined_df['query'].str.replace(' ', '_')
joined_df = joined_df.dropna(subset=['search_text'])

In [24]:
joined_df[joined_df['search_text'].isna()]

,query,query_type,content_id,total_click,position,score,search_text


In [25]:
corpus_df = film_df[['id', 'title', 'search_text']]
corpus_df.rename(columns={'id': '_id', 'search_text': 'text'}, inplace=True)
corpus_df.sample(2)

/tmp/ipykernel_2075106/4128656897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus_df.rename(columns={'id': '_id', 'search_text': 'text'}, inplace=True)


,_id,title,text
2705,5004,suami yang mudah berpaling dari istrinya,title: suami yang mudah berpaling dari istriny...
2633,2831,cinta nabrak tukang ketoprak,title: cinta nabrak tukang ketoprak\ndescripti...


In [26]:
query_df = joined_df[['query', 'search_text']]
query_df.rename(columns={'query': '_id', 'search_text': 'text'}, inplace=True)
query_df = query_df.drop_duplicates()
query_df = query_df.dropna()
query_df[query_df['_id'] == 'pertaruhan'].head()

/tmp/ipykernel_2075106/131488296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df.rename(columns={'query': '_id', 'search_text': 'text'}, inplace=True)


,_id,text
12562,pertaruhan,pertaruhan


In [27]:
training_test_df = joined_df[['query', 'content_id', 'score']]
training_test_df.rename(columns={'query': 'query-id', 'content_id': 'corpus-id'}, inplace=True)
training_test_df.dropna()
training_test_df['corpus-id'] = training_test_df['corpus-id'].astype(str)
training_test_df.sample(10)

/tmp/ipykernel_2075106/387614606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_test_df.rename(columns={'query': 'query-id', 'content_id': 'corpus-id'}, inplace=True)
/tmp/ipykernel_2075106/387614606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_test_df['corpus-id'] = training_test_df['corpus-id'].astype(str)


,query-id,corpus-id,score
11054,film_suster_el,4232,1
10999,film_kartun,5380,3
2436,319,710,5
4377,5342,4490,4
2976,3902,4073,4
1022,1967,519,2
3993,5026,4373,1
9830,anak_sekolahan,1397,2
13489,teuku_ryan,9607,1
4283,5270,5264,1


In [28]:
merged_df = pd.merge(training_test_df, corpus_df, left_on='corpus-id', right_on='_id', how='inner')
merged_df.drop(columns=['_id', 'title', 'text'], inplace=True)
merged_df.head()

,query-id,corpus-id,score
0,1003,1027,5
1,1003,831,4
2,1003,1559,3
3,1003,2415,2
4,1003,1299,1


In [29]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

In [30]:
train_df.sample(5)

,query-id,corpus-id,score
12936,siapa,479,1
10720,everest,7882,1
11567,ketos,3245,1
4159,5250,4003,5
6686,7647,1407,5


In [31]:
test_df.sample(5)

,query-id,corpus-id,score
3058,4021,1956,4
11921,majic_5,7576,2
10494,diantara_dua_cinta,8928,5
9081,9494,8264,3
884,1885,6640,2


In [32]:
corpus_df.to_json(f"data/{formatted_date_time}_corpus.json", orient='records', lines=True)
query_df.to_json(f"data/{formatted_date_time}_query.json", orient='records', lines=True)
train_df.to_csv(f"data/{formatted_date_time}_train.tsv", sep='\t', index=False)
test_df.to_csv(f"data/{formatted_date_time}_test.tsv", sep='\t', index=False)

In [35]:
trial_df = joined_df
trial_df.dropna(inplace=True)
trial_df['token_count'] = trial_df['search_text'].str.split().progress_apply(len)

  0%|          | 0/13769 [00:00<?, ?it/s]

In [36]:
trial_df['token_count'].sum()

742697